In [ ]:
%load_ext magic_duckdb


import fabduckdb
fabduckdb.register()


In [ ]:
%%dql 

CREATE OR REPLACE TABLE abc as SELECT * FROM range(2);
CREATE OR REPLACE TABLE def as SELECT * FROM range(3);

execute
        (WITH tablenames as (values ('abc'), ('def'))
                SELECT 'select * from ' || col0 FROM tablenames)
        using 'union all by name'

In [ ]:
import fabduckdb
import pandas as pd

fabduckdb.register()

fabduckdb.register_function("mydfcreator", lambda x: pd.DataFrame([{f"col{x}": x}]), generates_filepath=False)

In [ ]:
%%dql 

CREATE OR REPLACE TABLE xyz as SELECT * FROM range(4) t(x);

create or replace table abc as 
    (loop (select {{x}}, * from mydfcreator({{x}})) over (select x from xyz) using 'union')
;
select * from abc


In [ ]:
%%dql
--- Templated Loop Example using magic_duckdb
;
CREATE or REPLACE TABLE abc as 
    SELECT * FROM range(1,10,1) t(x)
;
CREATE or REPLACE TABLE def as 
    SELECT * FROM range(20,30,2) t(x)
;
LOOP
    (select {{x}}.x from {{x}}) 
OVER
    (select x from (values('abc'), ('def')) t(x))